# Tarea 10

ahora lo que procedemos a hacer es utilizar la derivación simétrica:

In [1]:
using SymPy #comenzaremos usando simbolos

Para la derivada hacia adelante usabamos cálculo de coeficientes de taylor a nuestra función $log(1+x)$ es importante recordar que por lo visto en la tarea anterior, esta última función se ha de sustituir por $arcsenh(x)$, buscando en la red hallamos que en SymPy se escribe "asenh(x)", ahora implementamos el código usado por el profesor, solo que sustituyendo $log(1+x)\rightarrow asinh(x);\ f(x+h)-f(x)\rightarrow f(x+h)-f(x-h)$:

In [6]:
function coeff_Taylor_sim(i)#implementamos las  rutinas cambiando lo señalado
x=Sym("x");
h=Sym("h");
N(subs(diff(asinh(x),x,i)/factorial(Float64(i)),x=>0))
end;

function coeff_Taylor_sim_rational(i)
    x=Sym("x");
    h=Sym("h");
    subs(diff(asinh(x),x,i)/factorial(i),x=>0)
end;

function finite_difference_sim(f)
    return (x,h)->f(x+h,h)-f(x-h,h)
end;

In [17]:
p=SymFunction("p")#hacemos una función simbólica

p

In [18]:
q(x,h)=p(x)#hacemos una función que ahora dependa de x y h

q (generic function with 1 method)

In [19]:
P=finite_difference_sim(q);#definimos P como la diferencia finita

In [20]:
P(Sym("x"),Sym("h"))#ahora solo hacemos que se vuelva una expresión simbólica

-p(-h + x) + p(h + x)

Con esto obtuvimos de manera correcta el operador de diferencia finita simétrica. Ahora haciendo la segunda aplicación de `finite_difference_symmetric`:

In [21]:
P=finite_difference_sim(P);
P(Sym("x"),Sym("h"))

-2*p(x) + p(-2*h + x) + p(2*h + x)

ahora procedemos a llevar a orden superior, pero antes recordemos:
$$\:\left.\frac{d}{d\xi}\right|_{\substack{\xi=x}}=\frac{1}{2h}\sum_{n=0}^{\infty}\frac{(-1)^{n}(2n)!}{4^{2n}(n!)^{2}(2n+1)}\delta_{h}^{\:2n+1}=\frac{1}{2h}\left(\delta_{h}-\frac{1}{24}\delta_{h}^{\:2}+\cdots\right)\ \ldots\ $$
Damos dos expresiones para dicha derivación debido a que tenemos dos funciones para los coeficientes de Taylor:

In [24]:
doc"""
esta función devuelve la derivada numérica simétrica como float de Julia usando coeficientes de taylor tipo float
introduzca (función,punto de aproximación,h,orden de aproximación)
"""
function Diff_order(f,x,h,o)
    g(x,h)=f(x)#hacemos dependencia en x y h
    d=0#iniciamos nuestra suma en cero
    for i in range(1,o)#con este for sumaremos terminos a der
        g=finite_difference_sim(g)
        d+=coeff_Taylor_sim(i)*g(x,h)/2^i
    end
    d/h
end

Diff_order

In [33]:
"""
esta función calcula la derivada simétrica como expresión de SymPy usando coeficientes de taylor racionales
introduzca (función,punto de aproximación,h,orden de aproximación)
"""
function Diff_order_rationalcoefficients(f,x,h,o)
    g(x,h)=f(x)
    d=0
    for i in range(1,o)
        g=finite_difference_sim(g)
        d+=coeff_Taylor_sim_rational(i)*g(x,h)/2^i
    end
    d/h
end

Diff_order_rationalcoefficients

__PRUEBA__:ahora procedemos a usar nuestra función

In [34]:
Diff_order(sin,10.0,0.01,6)#Aplicamos para la función sen(x) en el punto 10 con un incremento 0.01 y un orden 6

-0.8390715290763999

In [35]:
cos(10.0)#sabemos que la derivada de sen por eso evaluamos para comparar

-0.8390715290764524

para conocer como escala el error para comparar con la derivada hacia adelante usamos la función que arroja expresiones en SymPy

In [36]:
#Expresión de la derivada para un orden 5
A=Diff_order_rationalcoefficients(SymFunction("f"),Sym("x"),Sym("h"),6)

  3*f(-5*h + x)   25*f(-3*h + x)   75*f(-h + x)   75*f(h + x)   25*f(3*h + x) 
- ------------- + -------------- - ------------ + ----------- - ------------- 
       1280            768             128            128            768      
------------------------------------------------------------------------------
                                             h                                

  3*f(5*h + x)
+ ------------
      1280    
--------------
              

In [38]:
series(A,Sym("h"),0,9)#Expandimos en serie de Taylor:

                               /   7           \|               /   9         
                             6 |  d            ||             8 |  d          
                          5*h *|------(f(xi_1))||         25*h *|------(f(xi_1
                               |     7         ||               |     9       
/  d           \|              \dxi_1          /|xi_1=x         \dxi_1        
|-----(f(xi_1))||       + ----------------------------- + --------------------
\dxi_1         /|xi_1=x                112                             1152   

  \|              
  ||              
))||              
  ||              
  /|xi_1=x    / 9\
---------- + O\h /
                  

Para la derivada simétrica de orden 5 se tiene que el error escala como $h^{8}$. Se mejoró la aproximación respecto a la derivada hacia adelante propuesta en clase.